In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[2],2
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[-2, -2, 0]"
ng_refs,-,"[2, 3, 6]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.126104                    1  0.067547
0.000750        2  0.135432                    2  0.074385
0.001052        3  0.159256                    3  0.093665
0.001476        4  0.184836                    4  0.113229
0.002070        5  0.212358                    5  0.134419
0.002904        6  0.241026                    6  0.157198
0.004074        7  0.270651                    7  0.182720
0.005714        8  0.300452                    8  0.210666
0.008015        9  0.329539                    9  0.239969
0.011243       10  0.356778                   10  0.270039
0.015771       11  0.380712                   11  0.300197
0.022122       12  0.399897                   12  0.329419
0.031031       13  0.412769                   13  0.355901
0.043528       14  0.412162                   14  0.372190
0.061057       15  0.392992                   15  0.371231
0.085645       16  0.363204                   16  0.356148
0.120136       17  0.330525                   17  0.331178
0.168516       18  0.297977                   18  0.298952
0.236378       19  0.268807                   19  0.265442
0.331549       20  0.246716                   20  0.239720
0.465100       21  0.235437                   21  0.230380
0.652400       22  0.202341                   22  0.205723
0.915100       23  0.146597                   23  0.157055
1.283650       24  0.105410                   24  0.117234
1.800600       25  0.080712                   25  0.089523
2.525700       26  0.064747                   26  0.068814
3.542800       27  0.053107                   27  0.053831
4.969550       28  0.046117                   28  0.045152
6.970850       29  0.043784                   29  0.042528
9.778100       30  0.042211                   30  0.040143
13.715850      31  0.039664                   31  0.036404
19.239350      32  0.036538                   32  0.033126
26.987250      33  0.036171                   33  0.034006
37.855300      34  0.038999                   34  0.037244
53.100050      35  0.041637                   35  0.039173
73.887500      36  0.043847                   36  0.041280
97.662500      37  0.045387                   37  0.044230
121.437500     38  0.045595                   38  0.046276
145.212500     39  0.059984                   39  0.062774
168.987500     40  0.072645                   40  0.076954
192.762500     41  0.084191                   41  0.088071
216.537500     42  0.088756                   42  0.090417
240.312500     43  0.091147                   43  0.089714
264.087500     44  0.092560                   44  0.087949
287.862500     45  0.091337                   45  0.083636
311.637500     46  0.098337                   46  0.087939
335.412500     47  0.123935                   47  0.112244
359.187500     48  0.158493                   48  0.146088
382.962500     49  0.237489                   49  0.220552
406.737500     50  0.343994                   50  0.319718
430.512500     51  0.441217                   51  0.404675
454.287500     52  0.520795                   52  0.466426
478.062500     53  0.581304                   53  0.522344
501.837500     54  0.627116                   54  0.582743
525.612500     55  0.663064                   55  0.633841
549.387500     56  0.690957                   56  0.663451
573.162500     57  0.710582                   57  0.680850
596.937500     58  0.720252                   58  0.696467
620.712500     59  0.719377                   59  0.710004
644.487500     60  0.708396                   60  0.715731
668.262500     61  0.688736                   61  0.706894
692.037500     62  0.662415                   62  0.683310
715.812500     63  0.631507                   63  0.650036
739.587500     64  0.598349                   64  0.613291
763.362500     65  0.565270                   65  0.578145
78

# Fluxes by Layer

CRD                           CLIRAD             \
                        flug       fldg      fnetg       flug       fldg   
pressure    level                                                          
0.000000    1     -51.937341   0.000000 -51.937341 -51.863278   0.000002   
0.000624    2     -51.937349   0.000018 -51.937331 -51.863285   0.000015   
0.000876    3     -51.937353   0.000026 -51.937327 -51.863288   0.000020   
0.001229    4     -51.937358   0.000037 -51.937321 -51.863293   0.000029   
0.001723    5     -51.937364   0.000054 -51.937310 -51.863298   0.000041   
0.002417    6     -51.937373   0.000080 -51.937292 -51.863306   0.000060   
0.003391    7     -51.937384   0.000120 -51.937265 -51.863316   0.000088   
0.004757    8     -51.937399   0.000179 -51.937221 -51.863329   0.000131   
0.006672    9     -51.937419   0.000266 -51.937153 -51.863347   0.000196   
0.009359    10    -51.937444   0.000396 -51.937048 -51.863369   0.000294   
0.013128    11    -51.937474   0.000586 -51.936889 -51.863396   0.000442   
0.018415    12    -51.937511   0.000861 -51.936650 -51.863429   0.000663   
0.025830    13    -51.937553   0.001254 -51.936299 -51.863466   0.000989   
0.036232    14    -51.937597   0.001807 -51.935791 -51.863504   0.001466   
0.050823    15    -51.937639   0.002561 -51.935078 -51.863539   0.002144   
0.071291    16    -51.937676   0.003551 -51.934124 -51.863568   0.003073   
0.100000    17    -51.937705   0.004815 -51.932890 -51.863585   0.004302   
0.140271    18    -51.937709   0.006396 -51.931313 -51.863574   0.005871   
0.196760    19    -51.937664   0.008345 -51.929319 -51.863508   0.007806   
0.275997    20    -51.937519   0.010723 -51.926796 -51.863335   0.010124   
0.387100    21    -51.937167   0.013617 -51.923549 -51.862944   0.012888   
0.543100    22    -51.936342   0.017145 -51.919198 -51.862070   0.016273   
0.761700    23    -51.934951   0.020993 -51.913959 -51.860607   0.020137   
1.068500    24    -51.933297   0.024665 -51.908631 -51.858853   0.024091   
1.498800    25    -51.931528   0.028270 -51.903257 -51.856964   0.028179   
2.102400    26    -51.929651   0.032164 -51.897486 -51.854946   0.032562   
2.949000    27    -51.927705   0.036712 -51.890994 -51.852846   0.037364   
4.136600    28    -51.925876   0.042354 -51.883523 -51.850852   0.042944   
5.802500    29    -51.924316   0.049894 -51.874421 -51.849120   0.050123   
8.139200    30    -51.922903   0.060600 -51.862303 -51.847518   0.060294   
11.417000   31    -51.921638   0.075725 -51.845913 -51.846047   0.074412   
16.014700   32    -51.920976   0.096664 -51.824311 -51.845198   0.093391   
22.464000   33    -51.922076   0.125679 -51.796397 -51.846211   0.119714   
31.510500   34    -51.925638   0.168002 -51.757637 -51.849794   0.159741   
44.200100   35    -51.931076   0.232062 -51.699014 -51.855238   0.221175   
62.000000   36    -51.938309   0.327087 -51.611222 -51.862470   0.311012   
85.775000   37    -51.946660   0.458926 -51.487733 -51.870847   0.435659   
109.550000  38    -51.953364   0.593454 -51.359910 -51.877580   0.566971   
133.325000  39    -51.959785   0.728288 -51.231498 -51.884081   0.703812   
157.100000  40    -51.970148   0.907583 -51.062565 -51.894616   0.891156   
180.875000  41    -51.985963   1.127989 -50.857974 -51.910687   1.123977   
204.650000  42    -52.009398   1.388532 -50.620866 -51.934466   1.395817   
228.425000  43    -52.041307   1.670405 -50.370902 -51.967046   1.683068   
252.200000  44    -52.084251   1.970049 -50.114203 -52.011195   1.979906   
275.975000  45    -52.144330   2.290807 -49.853523 -52.073731   2.290160   
299.750000  46    -52.233898   2.637608 -49.596289 -52.168118   2.620118   
323.525000  47    -52.355822   3.036481 -49.319343 -52.297095   2.996784   
347.300000  48    -52.495269   3.524968 -48.970300 -52.444523   3.460359   
371.075000  49    -52.639248   4.115312 -48.523936 -52.596063   4.023373   
394.850000  50    -52.813755   4.958663 -47.855092 -52.778488   4.827006 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')